# Putting the R(etrieval) in RAG

In [ ]:
!pip install pinecone-client openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
import hashlib
from datetime import datetime

from tqdm import tqdm

In [ ]:
# Retrieve the Pinecone API key from user data
pinecone_key = userdata.get('PINECONE_API_KEY')

# Initialize the OpenAI client with the API key from user data
client = OpenAI(
    api_key=userdata.get('OPENAI_API_KEY')
)

# Define constants for the Pinecone index, namespace, and engine
INDEX_NAME = 'semantic-search-rag'  # The name of the Pinecone index
NAMESPACE = 'default'  # The namespace to use within the index
ENGINE = 'text-embedding-3-small'  # The embedding model to use (vector size 1,536)

# Initialize the Pinecone client with the retrieved API key
pc = Pinecone(
    api_key=pinecone_key
)


In [ ]:
# Function to get embeddings for a list of texts using the OpenAI API
def get_embeddings(texts, engine=ENGINE):
    # Create embeddings for the input texts using the specified engine
    response = client.embeddings.create(
        input=texts,
        model=engine
    )

    # Extract and return the list of embeddings from the response
    return [d.embedding for d in list(response.data)]

# Function to get embedding for a single text using the OpenAI API
def get_embedding(text, engine=ENGINE):
    # Use the get_embeddings function to get the embedding for a single text
    return get_embeddings([text], engine)[0]

# Test the functions by getting the length of a single embedding and a list of embeddings
len(get_embedding('hi')), len(get_embeddings(['hi', 'hello']))

(1536, 2)

In [ ]:
if INDEX_NAME not in pc.list_indexes().names():  # need to create the index
    print(f'Creating index {INDEX_NAME}')
    pc.create_index(
        name=INDEX_NAME,  # The name of the index
        dimension=1536,  # The dimensionality of the vectors for our OpenAI embedder
        metric='cosine',  # The similarity metric to use when searching the index
        spec=ServerlessSpec(
            cloud='aws',
            region='us-west-2'
        )
    )

# Store the index as a variable
index = pc.Index(name=INDEX_NAME)
index

In [ ]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
def my_hash(s):
    # Return the MD5 hash of the input string as a hexadecimal string
    return hashlib.md5(s.encode()).hexdigest()

my_hash('I love to hash it')

'ae76cc4dfd345ecaeea9b8ba0d5c3437'

In [ ]:
def prepare_for_pinecone(texts, engine=ENGINE, urls=None):
    # Get the current UTC date and time
    now = datetime.utcnow()

    # Generate vector embeddings for each string in the input list, using the specified engine
    embeddings = get_embeddings(texts, engine=engine)

    # Create tuples of (hash, embedding, metadata) for each input string and its corresponding vector embedding
    # The my_hash() function is used to generate a unique hash for each string, and the datetime.utcnow() function is used to generate the current UTC date and time
    responses = [
        (
            my_hash(text),  # A unique ID for each string, generated using the my_hash() function
            embedding,  # The vector embedding of the string
            dict(text=text, date_uploaded=now)  # A dictionary of metadata, including the original text and the current UTC date and time
        )
        for text, embedding in zip(texts, embeddings)  # Iterate over each input string and its corresponding vector embedding
    ]
    if urls and len(urls) == len(texts):
        for response, url in zip(responses, urls):
            response[-1]['url'] = url

    return responses


In [ ]:
texts = ['hi']

In [ ]:
_id, embedding, metadata = prepare_for_pinecone(texts)[0]

print('ID:  ',_id, '\nLEN: ', len(embedding), '\nMETA:', metadata)

ID:   49f68a5c8493ec2c0bf489821c21fc3b 
LEN:  1536 
META: {'text': 'hi', 'date_uploaded': datetime.datetime(2024, 5, 29, 19, 23, 38, 727917)}


In [ ]:
urls = ['fake.url']
_id, embedding, metadata = prepare_for_pinecone(texts, urls=urls)[0]

print('ID:  ',_id, '\nLEN: ', len(embedding), '\nMETA:', metadata)

ID:   49f68a5c8493ec2c0bf489821c21fc3b 
LEN:  1536 
META: {'text': 'hi', 'date_uploaded': datetime.datetime(2024, 5, 29, 19, 23, 39, 50436), 'url': 'fake.url'}


In [ ]:
def upload_texts_to_pinecone(texts, namespace=NAMESPACE, batch_size=None, show_progress_bar=False, urls=None):
    # Call the prepare_for_pinecone function to prepare the input texts for indexing
    total_upserted = 0
    if not batch_size:
        batch_size = len(texts)

    _range = range(0, len(texts), batch_size)
    for i in tqdm(_range) if show_progress_bar else _range:
        text_batch = texts[i: i + batch_size]
        if urls:
            url_batch = urls[i: i + batch_size]
            prepared_texts = prepare_for_pinecone(text_batch, urls=url_batch)
        else:
            prepared_texts = prepare_for_pinecone(text_batch)


        # Use the upsert() method of the index object to upload the prepared texts to Pinecone
        total_upserted += index.upsert(
            vectors=prepared_texts,
            namespace=namespace
        )['upserted_count']


    return total_upserted

In [ ]:
# Call the upload_texts_to_pinecone() function with the input texts
upload_texts_to_pinecone(texts)

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'default': {'vector_count': 1}},
 'total_vector_count': 1}

In [ ]:
texts

['hi']

In [ ]:
def query_from_pinecone(query, top_k=3, include_metadata=True):
    # get embedding from THE SAME embedder as the documents
    query_embedding = get_embedding(query, engine=ENGINE)

    return index.query(
      vector=query_embedding,
      top_k=top_k,
      namespace=NAMESPACE,
      include_metadata=include_metadata   # gets the metadata (dates, text, etc)
    ).get('matches')

In [ ]:
# test that the index is empty
query_from_pinecone('hello')

[{'id': '49f68a5c8493ec2c0bf489821c21fc3b',
  'metadata': {'date_uploaded': '2024-05-29T19:03:42.652561', 'text': 'hi'},
  'score': 0.80858922,
  'values': []}]

In [ ]:
import hashlib

def delete_texts_from_pinecone(texts, namespace=NAMESPACE):
    # Compute the hash (id) for each text
    hashes = [hashlib.md5(text.encode()).hexdigest() for text in texts]

    # The ids parameter is used to specify the list of IDs (hashes) to delete
    return index.delete(ids=hashes, namespace=namespace)

# delete our text
delete_texts_from_pinecone(texts)


{}

In [ ]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
base_url = 'https://faq.ssa.gov'
medicare_faqs = base_url + '/en-US/topic?id=CAT-01092'
print(medicare_faqs)

from bs4 import BeautifulSoup
import requests

# get all links from medicare_faqs
urls = []
r = requests.get(medicare_faqs)
soup = BeautifulSoup(r.content, 'html.parser')
for link in soup.find_all('a'):
    if 'href' in link.attrs:
        if link['href'].startswith('/') and 'article' in link['href']:
            urls.append(base_url + link['href'])

urls

https://faq.ssa.gov/en-US/topic?id=CAT-01092


['https://faq.ssa.gov/en-us/Topic/article/KA-01735',
 'https://faq.ssa.gov/en-us/Topic/article/KA-02713',
 'https://faq.ssa.gov/en-us/Topic/article/KA-02125',
 'https://faq.ssa.gov/en-us/Topic/article/KA-02166',
 'https://faq.ssa.gov/en-us/Topic/article/KA-02131',
 'https://faq.ssa.gov/en-us/Topic/article/KA-02995',
 'https://faq.ssa.gov/en-us/Topic/article/KA-02983',
 'https://faq.ssa.gov/en-us/Topic/article/KA-02137',
 'https://faq.ssa.gov/en-us/Topic/article/KA-02154',
 'https://faq.ssa.gov/en-us/Topic/article/KA-02113',
 'https://faq.ssa.gov/en-us/Topic/article/KA-02148',
 'https://faq.ssa.gov/en-us/Topic/article/KA-02989']

In [ ]:
texts = []
for url in tqdm(urls):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    body = soup.find('body').get_text()
    # CLEAN YOUR DATA HERE :)
    texts.append(body)

texts[0]

100%|██████████| 12/12 [00:32<00:00,  2.70s/it]


"\n\n\n\nYou’re offline. This is a read only version of the page.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n \n\n\n\n\n\n\nDo you need to submit W-2s to SSA? Business Services Online registration has changed!\n\n\n\n\n\n\n\n\n\n\nWhat should I do if I get a call claiming there's a problem with my Social Security number or account?\n\n\n\nSkip to main content Social Security Search  Menu  Español  Sign in\n\n\n\n\nFrequently Asked Questions\n\n\n\n\nLast Modified: \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFAQ Home\n\n\nTopics\n\n\r\n\t\t\t\t\tKA-01735\r\n\t\t\t\t\n\n\n\n\n\n Print\n\n\n\nHow do I get a replacement Medicare card? \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nViews: \n\n\n\nIf your Medicare card was lost, stolen, or destroyed, you can request a replacement online at Medicare.gov.\nYou can print an official copy of your card from your online Medicare account \nor call 1-800-MEDICARE (1-800-633-4227 TTY 1-877-486-2048) to order a replacement 

In [ ]:
BATCH_SIZE = 4
upload_texts_to_pinecone(texts, batch_size=BATCH_SIZE, urls=urls, show_progress_bar=True)

100%|██████████| 3/3 [00:04<00:00,  1.39s/it]


12

In [ ]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'default': {'vector_count': 12}},
 'total_vector_count': 12}

In [ ]:
 results = query_from_pinecone('I lost my medicare card', top_k=3)
 for result in results:
    print(result['metadata']['url'], result['score'], result['metadata']['text'][:50])

https://faq.ssa.gov/en-us/Topic/article/KA-01735 0.646217287 



You’re offline. This is a read only version of
https://faq.ssa.gov/en-us/Topic/article/KA-02713 0.500928938 



You’re offline. This is a read only version of
https://faq.ssa.gov/en-us/Topic/article/KA-02113 0.500682116 



You’re offline. This is a read only version of
